# 10장 모델설계 예제 - 폐암 수술 환자의 생존율 예측하기

In [10]:
# 딥러닝을 구동하는 데 필요한 케라스 함수를 불러온다.

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
   
# 필요한 라이브러리를 불러오가
import numpy as np
import tensorflow as tf

In [12]:
# 실행할 때마다 같은 결과를 출력하기 위해 설정. 근데 왜 하필 3임??
np.random.seed(3)
tf.random.set_seed(3)

# 데이터 불러오기
Data_set = np.loadtxt("/Users/KBS/Documents/study/python-basic/모두의딥러닝/ThoraricSurgery.csv", delimiter=",")

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장. why? Y는 클래스(정답)
X = Data_set[:, 0:17]
Y = Data_set[:, 17]

# 1. 입력, 은닉, 출력층 설정!


딥러닝 구조를 결정하는, 입력층과 은닉층 출력층을 설정하는 중요한 부분이다.

퍼셉트론 층들이 Sequential 함수를 통해 간단하게 구현이 된다.

Sequential() 함수를 model로 선언하고 model.add()를 하면 새로운 층이 만드렁짐.

근데 코드에는 model add가 두 개 있으니 층이 두 개인가? 하고 오해할 수 있다.

왜냐하면 다층퍼셉트론은 

입력 > 가중합 > 출력 이렇게 되어서 대충 동그라미 3개니까 층이 3개여야 하는거 아닌가 라고 생각할 수 있다. 

우선 model add로 만들어진 맨 마지막 층은 결과를 출력하는 "출력층"이다. 나머지는 모두 "은닉층"의 역할을 한다. 그럼 "입력층"은??

여기서 코드 model.add(Dense(30, input_dim=17, activation='relu'))를 보면 input_dim = 17 이 있다.

이는 입력데이터에서 몇 개의 값을 가져올지 정하는 것이다.

### 즉  keras는 입력층을 따로 만드는게 아니라!! 첫번째 은닉층 생성시 파라미터 값을 주어 입력층+은닉층 역할을 겸하는 것이다.

그 다음 30이라는 인자는 해당 층에 30개의 노드를 만들겠다는 뜻이다.

### 이렇게 각각의 층은 Dense라는 함수를 통해 구체적으로 그 구조 결정된다.

이렇게 은닉층의 각 노드는 17개의 입력 값에서 임ㅇ의의 가중치를 가지고 각 노드로 전송되어 활성화 함수를 만난다. 이때 활성화 함수를 지정해주는 것이 바로 activation이다.


In [13]:
# 딥러닝 구조를 결정
model = Sequential()
model.add(Dense(30, input_dim=17, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# 2. 모델 컴파일

model.compile부분은 앞서 지정한 모델이 효과적으로 구현될 수 있게 여러가지 환경 설정을 해주는 것,,,컴파일이란? 도대체 뭘까...


1. 먼저 손실함수를 지정한다. 손실(오차)함수를 최소화하는 방향으로 가중치(w)와 역치(k)를 수정하게 된다. 대표적인 오차함수 중 하나인 평균제곱오차를 사용했다, 이외 에도 교차분류엔트로피 등이 있다. MSE 계열은 수렴하기까지 속도가 많이 걸린다는 단점이 있다. 반면 교차 엔트로피 계열의 함수는 출력값에 로그를 취해서 오차가 커지면 수렴 속도가 빨라지고 오차가 작아지마녀 수렴 속도가 감소하게끔 만든 것

2. 훈련과정을 설정한다. 이 함수를 최적화하는 방향으로 학습이 일어나는 대표적인 함수 중 하나인 adam을 사용했다. 

3. 훈련과정을 모니터링하는 방식을 지정한다. accuracy를 지정하면 학습과정에서 정확도를 수집한다. 여기서 metrics함수는 모델이 컴파일 될 떄 모델 수행 결과를 나타나게끔 설정하는 방법이다. 정확도를 측정하기 위해 사용되는 테스트 샘플을 학습 과정에서 제외시킴으로써 과적합 문제를 방지함

In [15]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# 3. 모델 실행하기

모델을 정의하고 컴파일하고 나면 실행만 하면 된다. 아주 간단하다.

잠깐!! epochs가 뭔지 궁금했었는데 책에서 아주 잘 설명되고 있다.

학습프로세스가 모든 샘플에 대해 한 번 실행되는 것을 1epoch 라고 한다.

코드에서 epoch = 100으로 지정한 뜻은, 각 샘플이 처음부터 끝까지 100번 재사용될떄까지 실행을 반복하라는 뜻이다.

Batch_size는 샘플을 한 번에 몇 개씩 처리할지를 정하는 부분으로 batch_size = 10은 전체 470개의 샘플을 10개 씩 끊어서 집어넣으라는 뜻이 된다.

batch사이즈가 너무 크면 학습 속도가 느려지고, 너무 작으면 각 실행 값의 편차가 생겨서 전체 결과값이 불안정해질 수가 있다.

그리고 컴퓨터 메모리가 감당할 만큼 배치사이즈를 찾으라는데 찾는 방법은 책에 없다.

In [16]:
model.fit(X, Y, epochs=100, batch_size=10)

Epoch 1/100
47/47 [==============================] - 1s 2ms/step - loss: 0.1217 - accuracy: 0.8586
Epoch 2/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1122 - accuracy: 0.8578
Epoch 3/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1431 - accuracy: 0.8411
Epoch 4/100
47/47 [==============================] - 0s 2ms/step - loss: 0.1298 - accuracy: 0.8561
Epoch 5/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1246 - accuracy: 0.8567
Epoch 6/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1184 - accuracy: 0.8648
Epoch 7/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1244 - accuracy: 0.8521
Epoch 8/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1214 - accuracy: 0.8623
Epoch 9/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1131 - accuracy: 0.8614
Epoch 10/100
47/47 [==============================] - 0s 3ms/step - loss: 0.1130 - accuracy: 0.8732
Epoch 11/

Epoch 83/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1355 - accuracy: 0.8501
Epoch 84/100
47/47 [==============================] - 0s 1ms/step - loss: 0.0898 - accuracy: 0.9186
Epoch 85/100
47/47 [==============================] - 0s 972us/step - loss: 0.1213 - accuracy: 0.8603
Epoch 86/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1044 - accuracy: 0.8818
Epoch 87/100
47/47 [==============================] - 0s 961us/step - loss: 0.1109 - accuracy: 0.8684
Epoch 88/100
47/47 [==============================] - 0s 920us/step - loss: 0.1081 - accuracy: 0.8754
Epoch 89/100
47/47 [==============================] - 0s 990us/step - loss: 0.1262 - accuracy: 0.8498
Epoch 90/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1064 - accuracy: 0.8770
Epoch 91/100
47/47 [==============================] - 0s 956us/step - loss: 0.1092 - accuracy: 0.8734
Epoch 92/100
47/47 [==============================] - 0s 1ms/step - loss: 0.1204 - accurac

정확도가 0.8583이다
배치사이즈를 5로 바꿔보면 어떻게 될까?

In [17]:
model.fit(X, Y, epochs=100, batch_size=5)

Epoch 1/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1143 - accuracy: 0.8660
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.1262 - accuracy: 0.8511
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1213 - accuracy: 0.8723
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1211 - accuracy: 0.8617
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1188 - accuracy: 0.8702
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1180 - accuracy: 0.8723
Epoch 7/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1297 - accuracy: 0.8574
Epoch 8/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1176 - accuracy: 0.8702
Epoch 9/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1174 - accuracy: 0.8660
Epoch 10/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1144 - accuracy: 0.8745
Epoch 11/

94/94 [==============================] - 0s 1ms/step - loss: 0.1096 - accuracy: 0.8809
Epoch 84/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1124 - accuracy: 0.8638
Epoch 85/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1177 - accuracy: 0.8745
Epoch 86/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1237 - accuracy: 0.8574
Epoch 87/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1125 - accuracy: 0.8723
Epoch 88/100
94/94 [==============================] - 0s 2ms/step - loss: 0.1076 - accuracy: 0.8723
Epoch 89/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1165 - accuracy: 0.8723
Epoch 90/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1151 - accuracy: 0.8830
Epoch 91/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1175 - accuracy: 0.8681
Epoch 92/100
94/94 [==============================] - 0s 1ms/step - loss: 0.1163 - accuracy: 0.8723
Epoch 93/100


0.8787이라는 정확도가 나왔따. 로스도 약 0.02정도 줄었다.

## 아무튼 이번 10장에는 케라스함수를 호출해서 층을 만들고 실행까지 실습을 하였다. 우선 책을 2회독을 한 결과, 딥러닝 구동시, 이렇게 아주 간단한 기본적인 모델에서 분석 대상, 종류마다 함수들이 조금씩 더 추가된다. 일단 프로그래머스 강의에서는 함수의 파라미터들을 거의 95%정도 설명해주지 않아 이해하기 좀 힘들었는데, 차라리 책에 나와있는 쉬운 예제로 기초부터 차근차근 따라하다보면 이해하기 수월할 것으로 기대된다.